In [7]:
from __future__ import division
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import Laguerre_Gaussian as LG
import phaseScreenGeneration as PS
import CNN_Util as CU
import tensorflow as tf
from tensorflow import keras
import os
import pandas
import h5py
import json
from sklearn.metrics import classification_report, confusion_matrix
import scipy.ndimage as ndimage

##Noise

This was to see how the networks would react to a completly random input - i.e if there was a default class

In [8]:
def batch_generatorNoise(N_noise, ML_batch_size, N):
    """
    A function used to generate a batch of data for use in noise based testing.

    Args:
    N_noise = quanity of noise data
    ML_batch_size = self explanatory
    N = pixel width
    """
    batch=[]
    while True:
            for i in range(N_noise):
                batch.append(i)
                if len(batch)==ML_batch_size:
                    yield loadNoise(batch, N)
                    batch=[]
                    
def loadNoise(ids, N):
    """Function to generate noisy data of a given quanity.

    Args:
    ids = batch ids
    N = pixel width
    """
    #Initalising batchs to return
    X = np.zeros((0,N,N,1))
    Y = np.array([])
    x = np.zeros((1,N,N,1))
    
    for i in ids:
        
        x[0,:,:,0] = np.random.rand(N,N)
        x = x/np.amax(x)
        y = 0
        
        
        X = np.append(X, x, axis = 0)
        Y = np.append(Y,y)

    return np.array(X), np.array(Y)

In [ ]:
model_file_name = "3x3medium4ModeSNN"
data_source = "Data/3x3mediumTurb"

model = keras.models.load_model("CNN_D&G/" + model_file_name + 'Model.h5')

with open("CNN_D&G/" + model_file_name + "History.json", 'r') as f:
    history = json.load(f)
    
N = 256
modesUsed = 4
numTrainBatchPerMode = 4
batchSize = 50
numValBatchPerMode = 1
numBatchPerMode = 5
ML_BATCH_SIZE = 50

In [ ]:
N_noise = 200

ML_BATCH_SIZE = 50

validation_generator_noise = batch_generatorNoise(N_noise, ML_BATCH_SIZE, N)

Y_pred_noise = model.predict_generator(validation_generator_noise, (N_noise // ML_BATCH_SIZE))

print(Y_pred_noise.sum(0)/N_noise)

##Blanks

This was too see how the networks reacted to a blank array - i.e if there was a default class

In [ ]:
def batch_generatorBlank(N_blank, ML_batch_size, N):
    """
    A function used to generate a batch of empty arrays for use in testing.

    Args:
    N_blank = quanity of blank data
    ML_batch_size = self explanatory
    N = pixel width
    """
    batch=[]
    while True:
            for i in range(N_blank):
                batch.append(i)
                if len(batch)==ML_batch_size:
                    yield loadBlank(batch, N)
                    batch=[]

def loadBlank(ids, N):
    """Function to generate blank data of a given quanity.

    Args:
    ids = batch ids
    N = pixel width
    """
    #Initalising batchs to return
    X = np.zeros((0,N,N,1))
    Y = np.array([])
    x = np.zeros((1,N,N,1))
    
    for i in ids:
        
        x[0,:,:,0] = np.zeros((N,N), dtype=float)

        y = 0
        
        
        X = np.append(X, x, axis = 0)
        Y = np.append(Y,y)

    return np.array(X), np.array(Y)

In [ ]:
model_file_name = "strong32ModeCNN2"
data_source = "Data/3x3mediumTurb"

model = keras.models.load_model("CNN_D&G/" + model_file_name + 'Model.h5')
ML_BATCH_SIZE = 50

In [ ]:
N_blank = 200

validation_generator_blank = batch_generatorBlank(N_blank, ML_BATCH_SIZE, N)

Y_pred_blank = model.predict_generator(validation_generator_blank, (N_blank // ML_BATCH_SIZE))

print(Y_pred_blank.sum(0)/N_blank)

##Rotations

As the petal modes superpositions were roationally invarient over their projection rotation was not used to agument the data for training. However, in case a physical component was misaligned it was deemed of interest to know how roatated data was interpreted by each network

In [ ]:
def batch_generator_rotate(ids, ML_batch_size, numBatchPerMode, dataList, dataLabel, batchSize, angle):
    """
    A function used to generate a batch of images rotated at a certain angle.

    Args:
    ids = array of ids for each image
    ML_batch_size = self explanatory
    numBatchPerMode = the number of batch FILES for each mode to be used
    dataList = list of files the data is contained within
    dataLabel = labels for each id of true class
    batchSize = number of images per file
    angle = angle images are roatated in degrees
    """
    batch=[]
    while True:
            np.random.shuffle(ids) 
            for i in ids:
                batch.append(i)
                if len(batch)==ML_batch_size:
                    yield load_data_rotate(batch, numBatchPerMode, dataList, dataLabel, batchSize, angle)
                    batch=[]

def load_data_rotate(ids, numBatchPerMode, dataList, dataLabel, batchSize, angle):
    """
    A fucntion used to load data and its label from files and apply rotational agugmentation.
    
    Args:
    ids = array of ids for each image
    numBatchPerMode = the number of batch FILES for each mode to be used
    dataList = list of files the data is contained within
    dataLabel = labels for each id of true class
    batchSize = number of images per file
    angle = angle images are roatated in degrees
    """
    
    #Initalising batchs to return
    X = np.zeros((0,256,256,1))
    Y = np.array([])
    x = np.zeros((1,256,256,1))
    
    for i in ids:
        #Sorting out indices to get the correct files and preprocessing
        q = i%4
        k = np.floor(i/4)
        ql = np.floor(k/(numBatchPerMode*batchSize))
        j = np.floor((k-ql*numBatchPerMode*batchSize)/batchSize)
        insideIndex = k - (ql*numBatchPerMode + j) * batchSize
        
        #Getting the data
        Dat = np.load(dataList[int(j+ql*numBatchPerMode)], mmap_mode='r')
        x[0,:,:,0] = ndimage.rotate(Dat[:,:,int(insideIndex)], angle,reshape = False)
        y = dataLabel[int(i)]
        
        #Preprocessing
        if(q==3):
            x = np.flip(x,1)
            x = np.flip(x,2)
        elif(q==1 or q==2):
            x = np.flip(x,(q))
            
        x = x/np.amax(x)

        #Building batches
        X = np.append(X, x, axis = 0)
        Y = np.append(Y,y)

    return np.array(X), np.array(Y)

def list_labels_rotate_V(modesUsed, numValBatchPerMode, numBatchPerMode, batchSize, file_type):
    """
    Function used to generate rotated validation file names, label the data in them and provide ids.
    
    Args:
    modesUsed = number of diffrent OAM modes to be used (0 - modesUsed)
    numValBatchPerMode = the number of batch FILES for each mode to be used
    numBatchPerMode = number of batches per mode
    batchSize = number of images per file
    file_type = e.g "Data/weakTurb"
 
    Returns: arrays of
    DataList, DataLabel, ids
    """
    #Initialising lists
    DataList = []
    DataLabel = np.zeros(4*modesUsed*numValBatchPerMode*batchSize)
     
    #Loop to get names and labels
    for i in range(modesUsed):
        DataLabel[(i*numValBatchPerMode*batchSize*4):((i+3)*numValBatchPerMode*batchSize*4)] = i
        for j in range(numValBatchPerMode):
            DataList.append(file_type + str(i) + "Batch" + str(numBatchPerMode-1-j) + ".npy")
    ids = np.arange(4*modesUsed*numValBatchPerMode*batchSize)
    
    return DataList, DataLabel, ids

Tests

In [ ]:
angle_test = 0

ValidationDataList, ValidationDataLabel, idsV = list_labels_rotate_V(modesUsed, numValBatchPerMode, numBatchPerMode,
                                                                     batchSize, data_source)

validation_generator_rotate = batch_generator_rotate(idsV, ML_BATCH_SIZE, numBatchPerMode,
                                                     ValidationDataList, ValidationDataLabel, batchSize,
                                                     angle_test)

Y_pred_rotation = model.evaluate_generator(validation_generator_rotate, (len(idsV) // ML_BATCH_SIZE))

In [ ]:
model_file_name = "3x3extraStrong4ModeCNN1"
data_source = "Data/3x3extraStrongTurb"
modesUsed = 4

model = keras.models.load_model("CNN_D&G/" + model_file_name + 'Model.h5')

In [ ]:
step = 1
angles = np.arange(-90,90+step,step)
ValidationDataList, ValidationDataLabel, idsV = list_labels_rotate_V(modesUsed, numValBatchPerMode, numBatchPerMode,
                                                                     batchSize, data_source)
acc_rotation = np.zeros(len(angles))
count = 0
for angle_test in angles:
    
    validation_generator_rotate = batch_generator_rotate(idsV, ML_BATCH_SIZE, numBatchPerMode,
                                                     ValidationDataList, ValidationDataLabel, batchSize,
                                                     angle_test)

    Y_pred_rotation = model.evaluate_generator(validation_generator_rotate, (len((idsV)) // ML_BATCH_SIZE))
    
    acc_rotation[count] = Y_pred_rotation[1]
    count += 1
combined = np.zeros((len(angles),2))
combined[:,0] = angles
combined[:,1] = acc_rotation

plt.figure()
plt.plot(angles, acc_rotation)

Applying to a section of the dataset

In [ ]:
#Data Loaderer

data_list = ["AugmentationData/rotation4strongCNN1.npy", "AugmentationData/rotation4strongCNN2.npy",
            "AugmentationData/rotation4strongSNN.npy","AugmentationData/rotation8strongCNN1.npy",
            "AugmentationData/rotation8strongCNN2.npy","AugmentationData/rotation8strongSNN.npy",
            "AugmentationData/rotation16strongCNN1.npy","AugmentationData/rotation16strongCNN2.npy",
            "AugmentationData/rotation16strongSNN.npy","AugmentationData/rotation32strongCNN1.npy",
            "AugmentationData/rotation32strongCNN2.npy","AugmentationData/rotation32strongSNN.npy"]

model_list = ["3x3strong4ModeCNN1","3x3strong4ModeCNN2","3x3strong4ModeSNN",
             "3x3strong8ModeCNN1","3x3strong8ModeCNN2","3x3strong8ModeSNN",
             "3x3strong16ModeCNN1","3x3strong16ModeCNN2","3x3strong16ModeSNN",
             "3x3strong32ModeCNN1","3x3strong32ModeCNN2","3x3strong32ModeSNN"]

mode_list =[4,4,4,8,8,8,16,16,16,32,32,32]

In [ ]:
for i in range(len(model_list)-9):
    model = keras.models.load_model("CNN_D&G/" + model_list[i+9] + 'Model.h5')
    data_source = "Data/3x3strongTurb"
    modesUsed = mode_list[i+9]
    print(model_list[i+9])
    print("mode:" + str(modesUsed))
    step = 1
    angles = np.arange(0, 90+step,step)
    ValidationDataList, ValidationDataLabel, idsV = list_labels_rotate_V(modesUsed, numValBatchPerMode, numBatchPerMode,
                                                                         batchSize, data_source)
    acc_rotation = np.zeros(len(angles))
    count = 0
    for angle_test in angles:

        validation_generator_rotate = batch_generator_rotate(idsV, ML_BATCH_SIZE, numBatchPerMode,
                                                         ValidationDataList, ValidationDataLabel, batchSize,
                                                         angle_test)

        Y_pred_rotation = model.evaluate_generator(validation_generator_rotate, (len(idsV) // ML_BATCH_SIZE))

        acc_rotation[count] = Y_pred_rotation[1]
        count += 1
    combined = np.zeros((len(angles),2))
    combined[:,0] = angles
    combined[:,1] = acc_rotation
    np.save(data_list[i+9], combined)
    print(data_list[i+9])
    print(str(i+1) + " Completed")